In [98]:
pip install tensorflow  opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\ImageClassification\imageclasskernel\Scripts\python.exe -m pip install --upgrade pip' command.


In [99]:
import tensorflow as tf
import os

# 2. Remove dodgy images

In [100]:
import cv2
import imghdr

In [101]:
data_dir = 'data' 

In [102]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [103]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [104]:
import numpy as np
from matplotlib import pyplot as plt

In [105]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Veri yolu
data_path = "data"
labels = ['cloudy', 'dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime','shine', 'snow', 'sunrise']

image_paths = []
class_labels = []

# Her sınıf için dön
for label in labels:
    class_path = os.path.join(data_path, label)
    
    # Sınıfa ait resim dosyalarını listele
    for filename in os.listdir(class_path):
        img_path = os.path.join(class_path, filename)
        image_paths.append(img_path)
        class_labels.append(label)

# Veri setini eğitim ve test olarak böl
X_train_paths, X_test_paths, y_train, y_test = train_test_split(image_paths, class_labels, test_size=0.1, random_state=42)

# TensorFlow Dataset oluştur
def load_and_preprocess_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)  # JPEG formatı için
    img = tf.image.resize(img, [256, 256])
    img = tf.cast(img, tf.float32) / 255.0  # Görüntü normalizasyonu
    return img, label

# (image, label) çiftlerini içeren bir tuple oluştur
train_data = tf.data.Dataset.from_tensor_slices((X_train_paths, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test_paths, y_test))

# Resim ve etiketleri yükleyip normalize et
train_data = train_data.map(lambda x, y: load_and_preprocess_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.map(lambda x, y: load_and_preprocess_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle ve batch veri setini ayarla
batch_size = 32
train_data = train_data.shuffle(buffer_size=len(X_train_paths)).batch(batch_size)
test_data = test_data.batch(batch_size)

# Model ve cross-validation için devam edin


In [75]:
data_iterator = data.as_numpy_iterator()

In [76]:
batch = data_iterator.next()

# 4. Scale Data

In [57]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [58]:
# Veri setini normalize etme ve one-hot encoding uygulama
data = data.map(lambda x, y: (x / 255, tf.one_hot(y, depth=14)))


In [59]:
data.as_numpy_iterator().next() 

(array([[[[0.10673397, 0.28309447, 0.30667892],
          [0.11617982, 0.2893416 , 0.32279748],
          [0.11988454, 0.29711962, 0.334038  ],
          ...,
          [0.10571433, 0.25473392, 0.2586555 ],
          [0.10649654, 0.25551614, 0.26063928],
          [0.11784045, 0.26686007, 0.2784716 ]],
 
         [[0.09342831, 0.26380208, 0.28998497],
          [0.1031609 , 0.27605698, 0.30912513],
          [0.11462929, 0.28694853, 0.32496935],
          ...,
          [0.13543342, 0.2842735 , 0.28787196],
          [0.11846326, 0.2670233 , 0.2707151 ],
          [0.10123219, 0.24979225, 0.25348404]],
 
         [[0.08120404, 0.2434283 , 0.26723298],
          [0.08814482, 0.2556832 , 0.28284457],
          [0.10217764, 0.26965618, 0.3006764 ],
          ...,
          [0.15187366, 0.2973628 , 0.2951593 ],
          [0.10741134, 0.24858782, 0.24794155],
          [0.07132114, 0.2124976 , 0.2119471 ]],
 
         ...,
 
         [[0.05258789, 0.1270977 , 0.11925456],
          [0.06396

# 5. Split Data

In [62]:
from collections import Counter
class_labels = ['cloudy', 'dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']

# Train setindeki sınıfların örnek sayıları
train_class_counts = Counter([class_labels[np.argmax(label.numpy())] for _, label in train])

# Test setindeki sınıfların örnek sayıları
test_class_counts = Counter([class_labels[np.argmax(label.numpy())] for _, label in test])

print("Train seti sınıf örnek sayıları:")
print(train_class_counts)

print("\nTest seti sınıf örnek sayıları:")
print(test_class_counts)


Train seti sınıf örnek sayıları:
Counter({'fogsmog': 20, 'rime': 19, 'snow': 17, 'glaze': 17, 'hail': 14, 'rain': 14, 'frost': 12, 'lightning': 11, 'sunrise': 10, 'dew': 7, 'shine': 7, 'rainbow': 6, 'cloudy': 3})

Test seti sınıf örnek sayıları:
Counter({'hail': 4, 'rime': 4, 'glaze': 3, 'rain': 3, 'dew': 2, 'cloudy': 1, 'lightning': 1, 'fogsmog': 1, 'frost': 1, 'sunrise': 1, 'shine': 1})


# 6. Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Veri artırma işlemleri için ImageDataGenerator oluştur
datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=0.5,
    vertical_flip=0.005,
    channel_shift_range=0.10,
    brightness_range=(0.3, 0.3),
    #rescale=1./255,
    samplewise_center=True,
    samplewise_std_normalization=True
)

# Örnek bir görüntü yükleyin (bu, sizin gerçek görüntü yüklemeye uyarlamanız gereken bir yerdir)
image_path = 'data/snow/0853.jpg'
original_image = cv2.imread(image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Görüntüyü ekrana basmadan önce normalize et
original_image_normalized = original_image / 255.0

# Görüntüyü yeniden boyutlandır (ImageDataGenerator beklediği giriş boyutuna uygun olarak)
original_image_resized = tf.image.resize(original_image_normalized, (256, 256))

# Artırılmış görüntüyü oluştur
augmented_images = datagen.flow(tf.expand_dims(original_image_resized, axis=0), batch_size=1)
augmented_image = augmented_images.next()[0]

# Orijinal ve artırılmış görüntüyü göster
plt.subplot(1, 2, 1)
plt.imshow(original_image_resized.numpy())
plt.title('Orijinal Görüntü')

plt.subplot(1, 2, 2)
plt.imshow(augmented_image)
plt.title('Artırılmış Görüntü')

plt.show()


# 7. CNN Model

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,BatchNormalization

In [91]:
num_classes = 13 # Toplam sınıf sayısı

In [107]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()

model.add(BatchNormalization(input_shape=(256, 256, 3)))

model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.50))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 8. Train

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

hist = model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=32, validation_data=(X_val_fold, y_val_fold),callbacks=[tensorboard_callback, early_stopping])

In [ ]:
#hist = model.fit(train, epochs=30, validation_data=val,  callbacks=[tensorboard_callback, early_stopping])

# 9. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")

plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper right")
plt.show()

# 10. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Accuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
accuracy_metric = Accuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    acc.update_state(y, yhat)
    accuracy_metric.update_state(tf.argmax(y, axis=1), tf.argmax(yhat, axis=1))


test_accuracy = accuracy_metric.result().numpy()
print(f'Test Accuracy: {test_accuracy}')

# 11. Test

In [ ]:
# Görüntüyü okuma ve uygun formatta yükleme
img = cv2.imread('data/cloudy/cloudy1.jpg')

# Görüntü boş mu kontrolü
if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV, görüntüyü BGR formatında okur, bu nedenle RGB'ye dönüştürüyoruz

    # Görüntüyü gösterme
    plt.imshow(img)
    plt.show()
else:
    print("Görüntü yüklenemedi.")

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
img_path = 'data/cloudy/cloudy1.jpg'
img = cv2.imread(img_path)

if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resize = tf.image.resize(img, (256, 256))
    resize = np.expand_dims(resize / 255, 0)  

else:
    print(f"Error: Unable to read the image at {img_path}")


In [ ]:
class_names = ['cloudy', 'dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']
predictions = model.predict(resize)
predicted_class = np.argmax(predictions, axis=-1)

# Print some debug information
print(f'Predicted class indices: {predicted_class}')
if 0 <= predicted_class[0] < len(class_names):
    predicted_class_label = class_names[predicted_class[0]]
    print(f'Predicted class: {predicted_class_label}')
else:
    print('Error: Predicted class index is out of range.')

# Print class probabilities
for i, prob in enumerate(predictions[0]):
    class_label = class_names[i]
    print(f'Probability for class {class_label}: {prob}')


# 12. Save Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','weatherforecast18.h5'))

In [ ]:
new_model = load_model('D:\ImageClassification\models\weatherforecast18.h5')

In [ ]:
model.load_weights('D:\ImageClassification\models\weatherforecast18.h5')

# 13. Confussion Matrix

In [ ]:
pip install -U scikit-learn scipy matplotlib

In [ ]:
!pip install seaborn

In [ ]:
# Confusion Matrix oluşturma ve çizme
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

# Test veri seti üzerinde tahminlerin ve gerçek etiketlerin toplanması
y_true = []
y_pred = []

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    
    # Sınıf indekslerini listeler içinde ekleyin
    y_true.append([np.argmax(label) for label in y])
    y_pred.append([np.argmax(pred) for pred in yhat])

# Listeleri numpy dizilerine çevirme
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Confusion Matrix oluşturma
conf_mat = confusion_matrix(y_true, y_pred)

# Sınıf isimleri
class_names = ['cloudy', 'dew', 'fogsmog', 'frost', 'glaze', 'hail', 'lightning', 'rain', 'rainbow', 'rime', 'shine', 'snow', 'sunrise']

# Pandas DataFrame'e dönüştürme
conf_mat_df = pd.DataFrame(conf_mat, index=class_names, columns=class_names)

# Confusion Matrix'i çizme
plt.figure(figsize=(12, 10))
sns.heatmap(conf_mat_df, annot=True, fmt='g', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
